rooms_per_household : Nombre moyen de pièces par foyer, dérivé de total_rooms / households. Cela donne une idée de la taille moyenne des maisons.
bedrooms_per_room : Proportion de chambres par pièce totale, dérivé de total_bedrooms / total_rooms. Cela mesure l'utilisation des pièces comme chambres.
population_per_household : Moyenne de personnes par foyer, dérivé de population / households.
income_per_person : Revenu médian par personne, calculé comme median_income / population.
bedrooms_per_household : Proportion de chambres par foyer, dérivé de total_bedrooms / households.
rooms_x_income : Interaction entre rooms_per_household et median_income. Cela peut capturer la relation entre la taille des maisons et le niveau de revenu.
log_median_income : Logarithme du revenu médian pour réduire l'effet des valeurs extrêmes dans median_income.
log_population_per_household : Logarithme du ratio population_per_household pour lisser sa distribution.
Traitement des données manquantes :
total_bedrooms : Les valeurs manquantes sont remplacées par la médiane.
missing_bedrooms : Une colonne binaire est ajoutée pour indiquer si une valeur manquait dans total_bedrooms.
Encodage de ocean_proximity :
ocean_proximity est une colonne catégorique. Elle est transformée en colonnes binaires (ocean_proximity_INLAND, etc.) pour chaque catégorie.


In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Fonction pour créer de nouvelles caractéristiques
def create_new_features(df):
    df = df.copy()
    
    # Créer des ratios utiles
    df['rooms_per_household'] = df['total_rooms'] / df['households']
    df['bedrooms_per_room'] = df['total_bedrooms'] / df['total_rooms']
    df['population_per_household'] = df['population'] / df['households']
    df['income_per_person'] = df['median_income'] / df['population']
    df['bedrooms_per_household'] = df['total_bedrooms'] / df['households']

    # Colonnes supplémentaires
    df['rooms_x_income'] = df['rooms_per_household'] * df['median_income']
    df['log_median_income'] = np.log1p(df['median_income'])
    df['log_population_per_household'] = np.log1p(df['population_per_household'])
    
    # Gérer les valeurs manquantes
    df['total_bedrooms'].fillna(df['total_bedrooms'].median(), inplace=True)
    df['missing_bedrooms'] = df['total_bedrooms'].isnull().astype(int)
    
    # Encodage des catégories dans ocean_proximity
    ocean_categories = ['INLAND', 'NEAR BAY', 'NEAR OCEAN', '<1H OCEAN']
    for category in ocean_categories:
        df[f'ocean_proximity_{category}'] = (df['ocean_proximity'] == category).astype(int)
    df = df.drop('ocean_proximity', axis=1)
    
    return df

# Charger les données
train_data = pd.read_csv('./ynov-data/train_housing_train.csv')
valid_data = pd.read_csv('./ynov-data/train_housing_valid.csv')
test_data = pd.read_csv('./ynov-data/test_housing.csv')

# Appliquer la transformation des caractéristiques
train_data = create_new_features(train_data)
valid_data = create_new_features(valid_data)
test_data = create_new_features(test_data)

# Séparer les colonnes numériques et catégoriques
num_features = ['longitude', 'latitude', 'median_income', 'rooms_per_household', 
                'bedrooms_per_room', 'population_per_household', 'income_per_person',
                'bedrooms_per_household', 'rooms_x_income', 'log_median_income',
                'log_population_per_household']
cat_features = ['ocean_proximity_INLAND', 'ocean_proximity_NEAR BAY', 
                'ocean_proximity_NEAR OCEAN', 'ocean_proximity_<1H OCEAN']

# Préparation des jeux de données
X_train = train_data[num_features + cat_features]
y_train = train_data['median_house_value']

X_valid = valid_data[num_features + cat_features]
y_valid = valid_data['median_house_value']

X_test = test_data[num_features + cat_features]

# Pipeline pour le prétraitement des colonnes numériques
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Transformer toutes les colonnes via ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', 'passthrough', cat_features)  # Passthrough pour les colonnes catégoriques déjà encodées
    ])

# Créer le pipeline principal avec Linear Regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Entraîner le pipeline
pipeline.fit(X_train, y_train)

# Afficher les coefficients du modèle
print("Coefficients du modèle:", pipeline.named_steps['model'].coef_)
print("Intercept:", pipeline.named_steps['model'].intercept_)

# Validation croisée
cv_scores = cross_val_score(pipeline, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
print("\nCross-Validation MAE Scores:", -cv_scores)
print("Mean Cross-Validation MAE:", -cv_scores.mean())

# Évaluer sur le jeu de validation
valid_predictions = pipeline.predict(X_valid)
mae_valid = mean_absolute_error(y_valid, valid_predictions)
rmse_valid = mean_squared_error(y_valid, valid_predictions, squared=False)
r2_valid = r2_score(y_valid, valid_predictions)

print("\nÉvaluation sur l'ensemble de validation:")
print(f"MAE: {mae_valid:.2f}")
print(f"RMSE: {rmse_valid:.2f}")
print(f"R^2: {r2_valid:.2f}")

# Faire des prédictions sur le jeu de test
test_predictions = pipeline.predict(X_test)

# Créer un fichier de soumission
submission = pd.DataFrame({'id': test_data['id'], 'median_house_value': test_predictions})
submission.to_csv('./ynov-data/submit.csv', index=False)
print("\nFichier de soumission créé : './ynov-data/submit.csv'")


C:\Users\ouchaou\AppData\Local\Temp\ipykernel_7916\4084495860.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['total_bedrooms'].fillna(df['total_bedrooms'].median(), inplace=True)
C:\Users\ouchaou\AppData\Local\Temp\ipykernel_7916\4084495860.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

Coefficients du modèle: [ -63085.60842472  -66328.14409948   77102.79574953   35324.31383371
   25544.80022164    9341.99285753   -5076.38169176  -21268.19748893
  -10668.97878037    5336.65796     -29069.58594043 -155426.29664819
 -116138.53283289 -119715.60238037 -120524.52798783]
Intercept: 337141.4216565211

Cross-Validation MAE Scores: [47946.18668593 48409.69353854 49149.34763282 48508.52814827
 48460.13863023]
Mean Cross-Validation MAE: 48494.778927159976

Évaluation sur l'ensemble de validation:
MAE: 48730.57
RMSE: 68355.52
R^2: 0.66

Fichier de soumission créé : './ynov-data/submit.csv'


C:\Users\ouchaou\AppData\Local\pypoetry\Cache\virtualenvs\ynov-ix4t_1a0-py3.13\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
